# Challenge Tests

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.algorithms.ecg import (
    QPeakExtractionMartinez2004Neurokit,
)
from pepbench.algorithms.heartbeat_segmentation import HeartbeatSegmentationNeurokit
from pepbench.algorithms.icg import (
    BPointExtractionLozano2007LinearRegression,
)
from pepbench.algorithms.outlier_correction import (
    OutlierCorrectionDummy,
)
from pepbench.datasets import EmpkinsDataset
from pepbench.evaluation import PepEvaluationChallenge
from pepbench.pipelines import PepExtractionPipeline

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path()

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
print(empkins_base_path)

In [ ]:
result_path = root_path.joinpath("results/empkins_dataset_both_algorithms")
result_path.mkdir(exist_ok=True, parents=True)
result_path.resolve()

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

## Initialize Algorithms

### Heartbeat Segmentation

In [ ]:
heartbeat_segmentation_algo = HeartbeatSegmentationNeurokit()

### ECG - Q-Wave Onset

In [ ]:
q_wave_algo = QPeakExtractionMartinez2004Neurokit()

### ICG - B-Point

In [ ]:
b_point_algo = BPointExtractionLozano2007LinearRegression()

### Outlier Correction

In [ ]:
outlier_correction_algo = OutlierCorrectionDummy()

In [ ]:
subset = dataset_empkins.get_subset(participant=["VP_001", "VP_002"])

In [ ]:
pep_challenge = PepEvaluationChallenge(dataset=subset, validate_kwargs={"n_jobs": -1})
pipeline = PepExtractionPipeline(
    heartbeat_segmentation_algo=heartbeat_segmentation_algo,
    q_wave_algo=q_wave_algo,
    b_point_algo=b_point_algo,
    outlier_correction_algo=outlier_correction_algo,
    handle_negative_pep="nan",
    handle_missing_events="ignore",
)
pep_challenge.run(pipeline)
pep_challenge.results_as_df();

In [ ]:
pep_challenge.results_agg_total_

In [ ]:
pep_challenge.results_agg_mean_std_